# 延后初始化

到目前为止，我们忽略了建立网络时需要做的以下这些事情：

* 我们定义了网络架构，但没有指定输入维度。
* 我们添加层时没有指定前一层的输出维度。
* 我们在初始化参数时，甚至没有足够的信息来确定模型应该包含多少参数。

有些读者可能会对我们的代码能运行感到惊讶。
毕竟，深度学习框架无法判断网络的输入维度是什么。
这里的诀窍是框架的*延后初始化*（defers initialization），
即直到数据第一次通过模型传递时，框架才会动态地推断出每个层的大小。

在以后，当使用卷积神经网络时，
由于输入维度（即图像的分辨率）将影响每个后续层的维数，
有了该技术将更加方便。
现在我们在编写代码时无须知道维度是什么就可以设置参数，
这种能力可以大大简化定义和修改模型的任务。
接下来，我们将更深入地研究初始化机制。

## 实例化网络

首先，让我们实例化一个多层感知机。

此时，因为输入维数是未知的，所以网络不可能知道输入层权重的维数。
因此，框架尚未初始化任何参数，我们通过尝试访问以下参数进行确认。

接下来让我们将数据通过网络，最终使框架初始化参数。

一旦我们知道输入维数是20，框架可以通过代入值20来识别第一层权重矩阵的形状。
识别出第一层的形状后，框架处理第二层，依此类推，直到所有形状都已知为止。
注意，在这种情况下，只有第一层需要延迟初始化，但是框架仍是按顺序初始化的。
等到知道了所有的参数形状，框架就可以初始化参数。

## 小结

* 延后初始化使框架能够自动推断参数形状，使修改模型架构变得容易，避免了一些常见的错误。
* 我们可以通过模型传递数据，使框架最终初始化参数。

## 练习

1. 如果指定了第一层的输入尺寸，但没有指定后续层的尺寸，会发生什么？是否立即进行初始化？
1. 如果指定了不匹配的维度会发生什么？
1. 如果输入具有不同的维度，需要做什么？提示：查看参数绑定的相关内容。

### 练习一

1. 如果指定了第一层的输入尺寸，但没有指定后续层的尺寸，会发生什么？是否立即进行初始化？

&emsp;&emsp;如果指定了第一层的输入尺寸，但没有指定后续层的尺寸，可以正常运行。第一层会立即初始化，但其它层是直到数据第一次通过模型传递才会初始化。

In [1]:
import torch
from torch import nn

"""延后初始化"""
# 定义模型
net = nn.Sequential(nn.Linear(3, 4), nn.ReLU(), nn.LazyLinear(8))

# 尚未初始化
print("--- 数据未通过模型 ---")
print("第一层权重：", net[0].weight)
print("其它层的权重：", net[2].weight)

X = torch.rand(2, 3)
net(X)
# 已初始化
print("--- 数据第一次通过模型 ---")
print("第一层权重：", net[0].weight)
print("其它层的权重：", net[2].weight)

--- 数据未通过模型 ---
第一层权重： Parameter containing:
tensor([[ 0.0243,  0.5563,  0.5026],
        [-0.1443,  0.0025, -0.2089],
        [-0.5452,  0.2717,  0.4933],
        [-0.2920, -0.3691, -0.1432]], requires_grad=True)
其它层的权重： <UninitializedParameter>
--- 数据第一次通过模型 ---
第一层权重： Parameter containing:
tensor([[ 0.0243,  0.5563,  0.5026],
        [-0.1443,  0.0025, -0.2089],
        [-0.5452,  0.2717,  0.4933],
        [-0.2920, -0.3691, -0.1432]], requires_grad=True)
其它层的权重： Parameter containing:
tensor([[-0.0058, -0.2824, -0.2351,  0.3722],
        [ 0.1707,  0.4568,  0.2056,  0.4344],
        [-0.1049,  0.1305,  0.1953, -0.3996],
        [-0.0716,  0.1900,  0.1646, -0.3138],
        [ 0.0857,  0.3830, -0.3554,  0.2824],
        [ 0.1217,  0.3750, -0.2031,  0.0169],
        [-0.4419,  0.0478, -0.3999,  0.3104],
        [ 0.0398,  0.3117, -0.0463, -0.2907]], requires_grad=True)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


### 练习二

2. 如果指定了不匹配的维度会发生什么？

&emsp;&emsp;如果指定了不匹配的维度，会由于矩阵乘法时维度不匹配而报错。在下面的代码中便指定了不匹配的维度。

&emsp;&emsp;由于第一层nn.Linear(20, 256)的输入维度为20，所以输入数据 X 的最后一维必须为 20 才能与该层的权重矩阵相乘。

In [2]:
import torch
from torch import nn

# 定义模型
net = nn.Sequential(
    nn.Linear(20, 256), nn.ReLU(),
    nn.LazyLinear(128), nn.ReLU(),
    nn.LazyLinear(10)
)

X = torch.rand(2, 10)

print(X)

try:
    net(X)
except Exception as e:
    print(e)

tensor([[0.0012, 0.2153, 0.1139, 0.4348, 0.5692, 0.3747, 0.9115, 0.0711, 0.3245,
         0.2519],
        [0.4833, 0.7839, 0.1872, 0.7308, 0.9499, 0.1673, 0.0353, 0.4103, 0.6792,
         0.6314]])
mat1 and mat2 shapes cannot be multiplied (2x10 and 20x256)


### 练习三

3. 如果输入具有不同的维度，需要做什么？提示：查看参数绑定的相关内容。

&emsp;&emsp;如果输入维度比指定维度小，可以考虑使用padding填充；如果输入维度比指定维度大，可以考虑用 PCA 等降维方法，将维度降至指定维度。对于不同的维度，还可以添加一个额外的线性层，并将第一个线性层的权重与该层的权重绑定在一起。这样就可以解决维度不匹配的问题，并且保持模型的权重不变。注意，在下面代码中，我们假设第一个线性层的偏置项为零，因此不需要对其进行参数绑定。

In [3]:
import torch
import torch.nn as nn

# 定义模型
net = nn.Sequential(
    nn.Linear(20, 256), nn.ReLU(),
    nn.Linear(256, 128), nn.ReLU(),
    nn.Linear(128, 10)
)

X = torch.rand(2, 10)

try:
    net(X)
except Exception as e:
    print(e)

# 添加额外的线性层
extra_layer = nn.Linear(10, 256)
print(f"第一个线性层的维度：{net[0].weight.shape}")
print(f"额外的线性层的维度：{extra_layer.weight.shape}")

# 将第一个线性层与额外的线性层的权重进行绑定
net[0].weight = extra_layer.weight
print(f"第一个线性层的新维度：{net[0].weight.shape}")

# 使用新的输入（维度为20）调用模型
net(X)

mat1 and mat2 shapes cannot be multiplied (2x10 and 20x256)
第一个线性层的维度：torch.Size([256, 20])
额外的线性层的维度：torch.Size([256, 10])
第一个线性层的新维度：torch.Size([256, 10])


tensor([[-0.0300, -0.1461, -0.0879, -0.0359,  0.0522, -0.0396, -0.0155, -0.0326,
          0.0114,  0.0226],
        [-0.0481, -0.1210, -0.1034, -0.0464,  0.0291, -0.0096, -0.0130, -0.0273,
         -0.0020,  0.0556]], grad_fn=<AddmmBackward0>)